In [1]:
import pycuba as cb 
import numpy as np
import math
import lhapdf
import ROOT

%jsroot on

Welcome to JupyROOT 6.28/06


35.6627094422176
-5
-4
-3
-2
-1
1
2
3
4
5
21
LHAPDF 6.5.4 loading /home/pashax/Programs/LHAPDF-6.5.4/install/share/LHAPDF/NNPDF31_lo_as_0118/NNPDF31_lo_as_0118_0000.dat
NNPDF31_lo_as_0118 PDF set, member #0, version 1; LHAPDF ID = 315000


In [2]:
pi = ROOT.TMath.Pi()


def get_partonic_cs(s, t, u, alphas, flav1, flav2):
    result = 0.
    # dOmega -> dt
    J = 4*pi/s
    process_type = 0
    if abs(flav1) == abs(flav2) and (flav1 != 21 and flav2 != 21):
        process_type = 1 # qq / qbarqbar
        if flav1*flav2 < 0: # qqbar
            process_type = 2 
    elif abs(flav1) != abs(flav2) and (flav1 != 21 and flav2 != 21):
        process_type = 3 # non gluonic different flavour
        if flav1*flav2 < 0:
            process_type = 0 # qqbar'
    elif flav1 == 21 or flav2 == 21:
        process_type = 4 # halfgluonic
        if flav1 == 21 and flav2 == 21:
            process_type = 5 # gg

    #  no qqbar' interaction
    if process_type == 0: 
        return result

    if process_type == 1:
        # 51.7
        result += J*(alphas**2/9*s) * ((t**2+s**2)/(u**2) + (s**2+u**2)/(t**2) - (2*s**2)/(3*u*t))
    elif process_type == 2:
        # 51.4
        result += J*(alphas**2/9*s) * ((t**2+u**2)/(s**2))
        # 51.6
        result += J*(alphas**2/9*s) * ((t**2+u**2)/(s**2) + (s**2+u**2)/(t**2) - (2*u**2)/(3*s*t))
        # 51.9
        result += J*(8*alphas**2/27*s) * (t**2+u**2)* (1/(t*u) - 9/(4*s**2))
    elif process_type == 3:
        # 51.5
        result += J*(alphas**2/9*s) * ((s**2+u**2)/(t**2))
    elif process_type == 4:
        # 51.10
        result += J*(alphas**2/9*s) * (s**2+u**2)* (-1/(s*u) + 9/(4*t**2))
    elif process_type == 5:
        # 51.11
        result += J*(alphas**2/24*s) * (t**2+u**2)* (1/(t*u) - 9/(4*s**2))
        # 51.12
        result += J*(9*alphas**2/8*s) * (3 - u*t/(s**2) - s*u/(t**2) - s*t/(u**2))
    else:
        return result

    return result



-0.012217304763960308

In [6]:
## Getting a PDF member object
p = lhapdf.mkPDF("NNPDF31_lo_as_0118", 0)
## Gluon PDF querying at x=0.001, Q2=10000
#print(p.xfxQ2(21, 1e-3, 1e4))

s = 7000 # GeV

# define grid for pT and Delta y
# pTmin = 25

pts = np.arange(25, 100, 2)
del_y = np.arange(0, 20, 1)

def Integrand(ndim, xx, ncomp, ff, userdata):
    # access the current parameters
    dy,ybar = [xx[i] for i in range(ndim.contents.value)]
    # set the scale 
    pt = userdata
    # compute the result
    # define ranges and scale variables
    a1 = 0.
    b1 = 2.
    a2 = 0.
    b2 = 2.
    
    newdy = a1 + (b1-a1)*dy
    newybar = a2 + (b2-a2)*ybar

    result = 1
    #result = math.sin(newx)*math.sin(newy)*(b1-a1)*(b2-a2)
    # store the result (here only one component)
    ff[0] = result
    return 0

xx = [0., 0.]
ff = [0.]
NDIM = 2
NNEW = 1000
FLATNESS = 50.

data = 1

result = cb.Suave(Integrand, NDIM, NNEW, verbose=1, userdata=30)


LHAPDF 6.5.4 loading /home/pashax/Programs/LHAPDF-6.5.4/install/share/LHAPDF/NNPDF31_lo_as_0118/NNPDF31_lo_as_0118_0000.dat
NNPDF31_lo_as_0118 PDF set, member #0, version 1; LHAPDF ID = 315000

Iteration 1:  1000 integrand evaluations so far
[1] 1 +- 3.71222e-09  	chisq 0 (0 df)
